# 2. Get the Anecdote to Estimate Impact

Notebook to build all of the different stix objects associated with asking the reporting user about the Impact, and recording it through an Anecdote

We take v3.0 of the [Incident Core Extension](https://github.com/os-threat/cti-stix-common-objects/blob/main/extension-definition-specifications/incident-core/Incident%20Extension%20Suite.adoc) specification, with particular focus on the Sighting Extensions for Evidence, and combine it with the standard Stix approach. There are nine stages:

1. First setup the global parameters and retrieve the context memory
2. Create the Anecdote object and connect it to the Identity,
2. Add these objects to the `ObservedData` object
4. Create the `Sighting` object with the `SightingAnecdote` extension
5. Create the Impact object
5. Create the next Task and its Sequence objects
6. Connect the Sequence object to the previous sequence object
6. Add all of the objects to the existing Incident object
7. Save the context memory


Each time a block makes an object, and saves it as a json, this notebook will parse the object into an actual Stix object, so it can be bundled and printy printed (This step also verifies the objects are created correctly, and is cool).



## A. Load Imports

### A.1 StixORM Imports


In [ ]:
import sys
!{sys.executable} -m pip install stixorm
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### A.2 Relative Path Imports
https://stackoverflow.com/questions/76162459/jupyter-notebook-importerror-attempted-relative-import-with-no-known-parent-pac/77528726#77528726

In [ ]:
import sys
sys.path.append('../')
import os

### A.3 Relative Import of Blocks

In [ ]:
import json
from Block_Families.StixORM.SCO.EmailMessage.make_email_message import main as make_email_msg
from Block_Families.StixORM.SRO.Relationship.make_sro import main as make_sro
from Utilities.local_make_general import invoke_save_company_context_block, invoke_save_incident_context_block, invoke_get_from_company_block, invoke_get_from_user_block, invoke_move_unattached_to_other_block
from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block
from Utilities.local_make_sdo import (
    invoke_make_observed_data_block, invoke_make_indicator_block, invoke_make_event_block, invoke_make_sequence_block,
    invoke_make_task_block, invoke_make_incident_block, invoke_make_impact_block
)
from Utilities.local_make_sco import (
    invoke_make_email_addr_block, invoke_make_url_block, invoke_make_e_msg_block, invoke_make_anecdote_block
)
from conv import conv


context_base = "../Orchestration/generated/os-triage/context_mem/"
path_base = "../Block_Families/Objects/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_derived.json"
alert_data = {
    "from": "evil@northkorea.nk",
    "to": "naive@example.com",
    "reporter": "Naive Smith",
    "url": "https://www.northkorea.nk/we/are/mad/"
}
alert_details = {
    "alert_name": "Suspcious Email",
    "alert_source": "user-report",
    "alert_log": "I have found a suspicious email"
}
anecdote_data = {
    "to": "naive@example.com"
}



### A.4 Retrieve the TR User Account, the Email Address, and the TR Identity from the Context Memory

and the previous task sequence

In [ ]:
# 1. For Naive Smith, find the user account and the email. Plus get my own identity
reporter_name = alert_data["reporter"]
laptop_name = "HP Laptop 1"
TR_name = "Trusty Jones"
#
# 2. Setup variables and queries to get the objects from options
# get "HP Laptop 1"
context_type = {
    "context_type": "assets"
}
laptop_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": laptop_name,
        "comparator": "EQ"
    }
}
laptop_identity = invoke_get_from_company_block(laptop_ident_query, context_type, source_value=None, source_id=None)
# get reporter
context_type = {
    "context_type": "users"
}
reporter_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": reporter_name,
        "comparator": "EQ"
    }
}
reporter_identity = invoke_get_from_company_block(reporter_ident_query, context_type, source_value=None, source_id=None)
context_type = {
    "context_type": "me"
}
TR_ident_query = {
    "type" : "identity",
    "property": {
        "path": ["name"],
        "source_value": TR_name,
        "comparator": "EQ"
    }
}
TR_identity = invoke_get_from_user_block(TR_ident_query, context_type, source_value=None, source_id=None)
#
print(f"\nreporter_identity-->{reporter_identity}")
print(f"\nlaptop_identity-->{laptop_identity}")
print(f"\nTR_identity-->{TR_identity}")
#
# 3. save the objects to "unattached"
#
ident_results_obj_path = results_base + "/step0/harry1__ident.json"
ident_results_context_path = results_base  + "/step1/context/harry1__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path)
print(f" result->{result}")
ident_results_obj_path = results_base  + "/step0/me__ident.json"
ident_results_context_path = results_base + "/step1/context/me__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path)
print(f" result->{result}")
ident_results_obj_path = results_base  + "/step0/asset2__ident.json"
ident_results_context_path = results_base + "/step1/context/asset2__ident_context.json"
result = invoke_save_incident_context_block(ident_results_obj_path, ident_results_context_path)
print(f" result->{result}")

## B Make the Anecdote

In [ ]:
# 1. Setup the form paths and the results path
anecdote_form = "SCO/Anecdote/anecdote_on_impact.json"
results_path = "step2/step2"
# 2. Invoke the Make Anecdote block
anecdote = invoke_make_anecdote_block(anecdote_form, results_path, anecdote_reporter=reporter_identity)
# 3. Add the record to the in-session bundles and lists
anecdote_results_obj_path = results_base + results_path + "__anecdote.json"
anecdote_results_context_path = results_base + "/step2/anecdote_context.json"
result = invoke_save_incident_context_block(anecdote_results_obj_path, anecdote_results_context_path)
print(f" result->{result}")

## C. Collect the Elements of the Observation into an Observed-Data Object

Tbis then represents the potential phishing email as a group of elements

In [ ]:
# 1. Setup0 list of obsrvations
observed_ids = []
observed_ids.append(anecdote)
# 2. Setup path to form and results
obs_path ="SDO/ObservedData/observation-alert.json"
results_path ="step2/observation-anecdote.json"
# 3. Invoke the Make Observed Data Block
obs_2 = invoke_make_observed_data_block(obs_path, results_path, observation=observed_ids)
# 4. Add the record to the in-session bundles and lists
observation_results_obj_path = results_base + "/step2/observation-anecdote.json"
observation_results_context_path = results_base + "/step2/observation_anecdote_context.json"
result = invoke_save_incident_context_block(observation_results_obj_path, observation_results_context_path)
print(f" result->{result}")

## D. Create the Anecdote Sighting

Tbis then represents the Anecdote on Impact from the reporting user

### D.1 Create the Sighting
Connecting the:
- observed-data object
- the reporter identity as the sighted object
- the laptop identity as the location

In [ ]:
# 1. Setup path to form and results
sighting_data_path ="SRO/Sighting/sighting_anecdote.json"
results_path ="step2/sighting_anecdote.json"
# 2. Setup the SDO sighted, the Observed-Data that was observed with generated objects,
#                        then the identity object from the context storage (note the slight difference in indexing #
observed_list = [obs_2]
sighted = reporter_identity
TR_where_list = [TR_identity]
# 2. Invoke the Make Observed Data Block
sight2 = invoke_sighting_block(sighting_data_path, results_path, observed=observed_list, sighted=sighted, where=TR_where_list)
# 3. Add the record to the in-session bundles and lists
sighting_results_obj_path = results_base + "/step2/sighting_anecdote.json"
sighting_results_context_path = results_base + "/step2/sighting_anecdote_context.json"
result = invoke_save_incident_context_block(sighting_results_obj_path, sighting_results_context_path)
print(f" result->{result}")

### D.2 Move Unattached to Other

In [ ]:
# total_list = observed_ids
# total_list.append(obs_2)
# total_list.append(TR_identity)
# total_list.append(sight2)
# total_list.append(reporter_identity)

# obs_context_move_path = results_base + "step2/context/context_move.json"
# obs_context_move_results = results_base + "step2/context/context_move_results.json"
# result = invoke_move_unattached_to_other_block(obs_context_move_path, obs_context_move_results, total_list)

## E. Create the Impact Objects

Tbis Impact represents the effect reported by the user in the Anecdote

In [ ]:
# 1. Setup path to form and results
impact_path ="SDO/Impact/anecdote_impact.json"
results_path ="step2/impact_anecdote.json"
# 2. Setup the number of assets impacted
numbers = {"computers-mobile": 1}
impacted_refs = [laptop_identity["id"]]
# 2. Invoke the Make Observed Data Block
impact_1 = invoke_make_impact_block(impact_path, results_path, impacted_entity_counts=numbers, impacted_refs=impacted_refs, superseded_by_ref=None)
# 3. Add the record to the in-session bundles and lists
impact_results_obj_path = results_base + results_path
impact_results_context_path = results_base + "/step2/impact_anecdote_context.json"
result = invoke_save_incident_context_block(impact_results_obj_path, impact_results_context_path)
print(f" result->{result}")

## F. Create the Next Task Object

Next step is to investigate the Exchange server to see who else got the email

### F.1 Create the Task Object

In [ ]:
# 1. Setup path to form and results
task_data_path ="SDO/Task/task_anecdote.json"
results_path ="step2/task_anecdote.json"
# 2. Invoke the Make Observed Data Block
task_2 = invoke_make_task_block(task_data_path, results_path, changed_objects=None)
# 3. Add the record to the in-session bundles and lists
task_results_obj_path = results_base + "/step2/task_anecdote.json"
task_results_context_path = results_base + "/step2/task_anecdote_context.json"
result = invoke_save_incident_context_block(task_results_obj_path, task_results_context_path)
print(f" result->{result}")

### F.2 Create the Sequence Object

In [ ]:
# 1. Setup path to form and results
sequence_data_path ="SDO/Sequence/sequence_anecdote.json"
results_path ="step2/sequence_task_anecdote.json"
# 2. Setup the Sequence Object for the Event
#
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_2["id"]
# 2. Invoke the Make Observed Data Block
seq_T_2 = invoke_make_sequence_block(sequence_data_path, results_path, step_type=step_type, sequence_type=sequence_type, sequenced_object=sequenced_object, on_completion=None, on_success=None, on_failure=None, next_steps=None)
# 3. Add the record to the in-session bundles and lists
sequence_results_obj_path = results_base + "/step2/sequence_task_anecdote.json"
sequence_results_context_path = results_base + "/step2/sequence_anecdote_context.json"
result = invoke_save_incident_context_block(sequence_results_obj_path, sequence_results_context_path)
print(f" result->{result}")

### F.3 Connect the Sequence to the Previous one